## 딥러닝 실습 과제 1주차 - 데이터 전처리

다음 세 가지 활동을 해봅시다.

01. **이미지 & 레이블 로드**: JSON 파일과 이미지 데이터를 PyTorch Dataset 형식으로 변환
02. **이미지 전처리**: 크기 조정, 정규화
03. **학습/검증/테스트 데이터 분할**


TTTDataset.zip을 불러와 문제에서 요하는 코드를 구현하세요.

💡 **데이터 구조**  
- **`image_black`** : 이미지 데이터  
- **`labels`** : 타겟 데이터  

In [1]:
import torch
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
from PIL import Image
import json
from torch.utils.data import random_split
import glob
import os

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
from sklearn.model_selection import train_test_split

## 00. 클래스
정의한 클래스를 이용해 실행해 주세요.

In [4]:
image_dir = '/content/drive/My Drive/KHS/TTTDataset/image_black'
label_dir = '/content/drive/My Drive/KHS/TTTDataset/labels'

image_paths = sorted(glob.glob(os.path.join(image_dir, "*.jpg")) +
                     glob.glob(os.path.join(image_dir, "*.JPG")))
label_paths = sorted(glob.glob(os.path.join(label_dir, "*.json")))

print(f"이미지 파일 수: {len(image_paths)}")
print(f"라벨 파일 수: {len(label_paths)}")

이미지 파일 수: 453
라벨 파일 수: 453


In [5]:
class TTTDataset(Dataset):
    def __init__(self, image_paths, label_paths, transform=None):
        """
        틱택토 데이터셋을 PyTorch Dataset 형태로 변환.
        :param image_paths: 이미지 파일 경로 리스트
        :param label_paths: 레이블 JSON 파일 경로 리스트
        :param transform: 이미지 전처리 변환
        """
        self.image_paths = image_paths
        self.label_paths = label_paths
        self.transform = transform
        self.data = self._load_data()


    def _load_data(self):
        """ 이미지 & 레이블 로드 """
        data = []
        for img_path, lbl_path in zip(self.image_paths, self.label_paths):
            # 이미지를 흑백(Grayscale)로 변환
            image = Image.open(img_path).convert("L")  # "RGB" 대신 "L" 사용

            # JSON 레이블 로드
            with open(lbl_path, 'r') as f:
                labels = json.load(f)

            # 레이블을 숫자로 변환 (O=1, X=-1, blank=0)
            label_tensor = torch.tensor(
                [1 if v == "O" else -1 if v == "X" else 0 for v in labels.values()],
                dtype=torch.float32
            )
            data.append((image, label_tensor))

        return data


    def __len__(self):
        """ 데이터셋 크기 반환 """
        return len(self.data)


    def __getitem__(self, idx):
        """ 데이터셋에서 idx 번째 샘플(이미지 & 레이블)을 가져오는 역할 """
        image, label = self.data[idx]

        if self.transform:
            image = self.transform(image)

        return image, label

## 01. 이미지 & 레이블 로드: JSON 파일과 이미지 데이터를 PyTorch Dataset 형식으로 변환

In [6]:
class TTTDataset(Dataset):
    def __init__(self, image_paths, label_paths, transform=None):

      self.image_paths = image_paths
      self.label_paths = label_paths
      self.transform = transform
      self.data = self._load_data()

    def get_image_paths(image_dir):
      valid_extentions = ('.jpg', '.JPG')
      image_paths = [os.path.join(image_dir, f) for f in os.listdir(image_dir) if f.endswith(valid_extentions)]
      return image_paths

    def get_label_paths(label_dir):
      valid_extentions = ('.json')
      label_paths = [os.path.join(label_dir, f) for f in os.listdir(label_dir) if f.endswith(valid_extentions)]
      return label_paths

    def _load_data(self):
        """ 이미지 & 레이블 로드 """
        data = []
        for img_path, lbl_path in zip(self.image_paths, self.label_paths):
            # 이미지를 흑백(Grayscale)로 변환
            image = Image.open(img_path).convert("L")  # "RGB" 대신 "L" 사용

            # JSON 레이블 로드
            with open(lbl_path, 'r') as f:
                labels = json.load(f)

            # 레이블을 숫자로 변환 (O=1, X=-1, blank=0)
            label_tensor = torch.tensor(
                [1 if v == "O" else -1 if v == "X" else 0 for v in labels.values()],
                dtype=torch.float32
            )
            data.append((image, label_tensor))

        return data

    def __len__(self):
      return len(self.data)


    def __getitem__(self, idx):
        """ 데이터셋에서 idx 번째 샘플(이미지 & 레이블)을 가져오는 역할 """
        image, label = self.data[idx]

        if self.transform:
            image = self.transform(image)

        return image, label

## 02. 이미지 전처리: 크기 조정, 정규화

In [7]:
transform = transforms.Compose([
    transforms.Resize((28, 28)),  # 이미지 크기를 28x28로 조정
    transforms.ToTensor(),  # 이미지를 PyTorch 텐서로 변환
    transforms.Normalize((0.5,), (0.5,))  # 정규화: 평균 0.5, 표준편차 0.5
])

## 03. 학습/검증/테스트 데이터 분할

In [8]:
# dataset 생성
dataset = TTTDataset(image_paths, label_paths, transform=transform)

total_size = len(dataset)
train_size = int(0.7*total_size)
val_size = int(0.15*total_size)
test_size = total_size - train_size - val_size

# 분할
train_dataset, val_dataset, test_dataset = random_split(dataset, [train_size, val_size, test_size])

print(f"Train dataset size: {len(train_dataset)}")
print(f"Validation dataset size: {len(val_dataset)}")
print(f"Test dataset size: {len(test_dataset)}")

Train dataset size: 317
Validation dataset size: 67
Test dataset size: 69


In [10]:
# train // test
train_img_paths, test_img_paths, train_lbl_paths, test_lbl_paths = train_test_split(
    image_paths,
    label_paths,
    test_size=0.2,  # 20%를 테스트 세트로
    random_state=42  # 랜덤 시드를 고정하여 결과 재현 가능
)

# test // val
val_img_paths, test_img_paths, val_lbl_paths, test_lbl_paths = train_test_split(
    test_img_paths,
    test_lbl_paths,
    test_size=0.5,  # 20% 중 50%를 검증 세트로 나누므로, 전체 데이터의 10%가 검증 세트로
    random_state=42
)

# 데이터셋 객체 생성
train_dataset = TTTDataset(train_img_paths, train_lbl_paths, transform=transform)
val_dataset = TTTDataset(val_img_paths, val_lbl_paths, transform=transform)
test_dataset = TTTDataset(test_img_paths, test_lbl_paths, transform=transform)


print(f"Train dataset size: {len(train_dataset)}")
print(f"Validation dataset size: {len(val_dataset)}")
print(f"Test dataset size: {len(test_dataset)}")


Train dataset size: 362
Validation dataset size: 45
Test dataset size: 46
